In [1]:
%%configure -f

{"conf":
     {"spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.pyspark.virtualenv.bin.path":"/home/hadoop/epidemics-venv/bin",
      "spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.dynamicAllocation.enabled":"true",
      "spark.executor.memory":"16g",
      "spark.driver.memory":"16g",
      "spark.executor.cores":"6",
      "spark.driver.cores":"6",
      "spark.driver.maxResultSize":"20g",
      "livy.server.session.timeout-check" :"false"}
}

In [2]:
import pandas as pd
import geopandas as gpd
import json
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, shape, box
from shapely.ops import unary_union, transform

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, ArrayType

import daphme as dm
from daphme import cleaning

from sedona.spark import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1703545555913_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Example1").\
    getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Intro to DAPHME

In this script, we will demonstrate some of DAPHME's functionalities for analyzing human mobility data. Our analysis will focus on the month of February for the city of Philadelphia. Using a dataset that originally contains [60000] users. 

In our analysis, we will perform three different kinds of operations, which are common in the analyses in these types of data. 
* **Importing**, which optionally converts the datetime to a specified timezone and project the coordinates into ESPG:3857 (which has units in meters and might be appropriate for a local analysis not too far from the equator). 
* **Coarse Filtering**, where we subset to users to make computations for tractable. In particular, we filter out users with very few days/hours with activity, and keep TODO [users with "sufficient" pings in the area]. We also filter to users with activity in a given time period.
* **Persisting**

### Load Pings

Daphme can work with a single file in a number of formats (csv, parquet, txt) but its true utility comes when working with partitions of the data, which are often organized in a directory. In this example, we specify the path of the files to be loaded.

In [29]:
gravy_raw = spark.read.parquet("s3://phl-pings/gravy/")
#gravy_raw = spark.read.parquet("s3://phl-pings/gravy/part_0.snappy.parquet")

gravy_raw = gravy_raw. \
    withColumn("latitude", F.col("geohashlatitude").cast(DoubleType())). \
    withColumn("longitude", F.col("geohashlongitude").cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
gravy_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- grid: string (nullable = true)
 |-- geohashlatitude: string (nullable = true)
 |-- geohashlongitude: string (nullable = true)
 |-- geohashnine: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- timezone: string (nullable = true)
 |-- ipaddress: string (nullable = true)
 |-- forensicflag: long (nullable = true)
 |-- devicetype: string (nullable = true)
 |-- recordcount: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [ ]:
#cuebiq_df = spark.read.options(header='True', inferSchema='True', delimiter=',').parquet("s3://phl-pings/cuebiq-jan-mar/")
#cuebiq_df.show()

### Convert UTC to local datetime

In [30]:
gravy_df = cleaning.to_local_time(gravy_raw, 'America/New_York', epoch_unit='milliseconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Coarse filter pings to Philadelphia

Takes in a geometry and filters to pings within the given geometry

In [32]:
# Load in a bounding box for Philadelphia

import boto3
from io import BytesIO

s3 = boto3.client('s3')
bucket_name = 'upenn-seas-wattscovid19lab'
object_key = 'paco/geometry/Census_Tracts_2010.geojson'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)
geojson_data = json.load(BytesIO(obj['Body'].read()))

features = geojson_data['features']

polygons = [shape(feature["geometry"]).buffer(0) for feature in features if feature["properties"]["GEOID10"][:5] == "42101"]
phila_poly = unary_union(polygons).buffer(0.0015).simplify(0.0015)
phila_box = box(*phila_poly.bounds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
phila_box_wkt = phila_box.wkt

gravy_df_filtered = cleaning.coarse_filter(gravy_df, phila_box_wkt, spark, id_col="grid")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Convert coordinates to mercator in meters

In [37]:
gravy_df_filtered = cleaning.to_mercator(gravy_df_filtered, spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Persist

In [40]:
gravy_df_clean = gravy_df_filtered.select(
    'grid', 
    'local_timestamp', 
    'mercator_coord', 
    'x',
    'y',
    'in_geo',
    'date', 
    'date_hour', 
    'day_of_week'
).withColumnRenamed("grid", "identifier")
gravy_df_clean.write.partitionBy("date").option("header", "true").mode("overwrite").parquet("s3://phl-pings/gravy_clean/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
#gravy_df_clean.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------------+-------------------+------------------+------+----------+-------------+-----------+
|          identifier|    local_timestamp|      mercator_coord|                  x|                 y|in_geo|      date|    date_hour|day_of_week|
+--------------------+-------------------+--------------------+-------------------+------------------+------+----------+-------------+-----------+
|00018c51-ff4a-3fb...|2019-11-18 11:53:12|POINT (4871437.66...| -8356676.361526982| 4871437.666248344|  true|2019-11-18|2019-11-18 11|          2|
|00018c51-ff4a-3fb...|2019-11-18 11:55:11|POINT (4860087.72...| -8370869.819242106| 4860087.720346581|  true|2019-11-18|2019-11-18 11|          2|
|0001b67f-8b1e-33d...|2020-06-19 15:25:31|POINT (4859015.82...| -8367196.053406948| 4859015.823786038|  true|2020-06-19|2020-06-19 15|          6|
|00028c4e-4008-374...|2020-02-14 16:38:56|POINT (4855894.02...| -8350910.123223381| 4855894.026816804|  true|2020-02-1